# Assignment: Data Wrangling
## `! git clone https://github.com/DS3001/wrangling`
## Do Q2, and one of Q1 or Q3.

**Q1.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?

  This paper is about an important component of data cleaning: data tidying.

  2. Read the introduction. What is the "tidy data standard" intended to accomplish?

  The "tidy data standard" is meant to standardize data cleaning, making data cleaning easier. Data scientists no longer need to start the data cleaning process from scratch each time.

  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."

  The first sentence means that tidy datasets usually follow a standard format or structure, which makes them easy to work with and analyze. On the contrary, messy datasets vary widely, each presenting unique challenges when working with them.

  The second sentence suggests that while identifying observations and variables in a specific dataset is often straightforward, creating a clear definition for what constitutes the two across all datasets can be complex and nuanced.

  4. Read Section 2.2. How does Wickham define values, variables, and observations?

  Values are numeric or categorical, a variable is a collection of values that measure the same attribute, and an observation is a collection of values that measure it.

  5. How is "Tidy Data" defined in section 2.3?

  Each variable is a column, each observation is a row, and each type of observational unit is a table. Data is messy if not tidy.

  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?

  - Column headers are values, not variable names
  - Multiple variables are stored in one column
  - Variables are stored in both rows and columns
  - Multiple types of observational units are stored in the same table
  - A single observational unit is stored in multiple tables

  The data in Table 4 are messy because the column headers are values (income ranges) and not variable names.

  Melting a dataset is the process of converting column-value variables into rows.

  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?

  Table 11 is messy because the column headers are once again values (days). Table 12 then melts those day value columns into a single variable 'date'. Further more, the element column in table 11 contains variable names and not values. Tmax and tmin -- the "values" under the element column are actually names of variables, and not the actual values themselves. Whereas the tidy version of table 12 show the actual values of tmin and tmax for each record.

  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

  The "chicken-and-egg" problem with focusing on tidy data is that if tidy data is only as useful as the tools that work with it, then tidy tools will be inextricably linked to tidy data. This makes it easy to get stuck in a local maxima where independently changing data structures or data tools will not improve the workflow.

  He hopes that in the future, tidy can create a bigger and more robust ecosystem for ideas and tools in data science.

**Q2.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the `./data/sharks.csv` data covered in the lecture, clean the "Type" variable as well as you can, and explain the choices you make.
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
df = pd.read_csv('./airbnb_hw.csv',low_memory=False)
df.head()

In [ ]:
price = df['Price']
price.unique()

In [ ]:
price=price.str.replace(',','') # Remove the commas
price = pd.to_numeric(price,errors='coerce') # Price to numeric instead of string
print(price.unique(),'\n')
print('Total missing: ',sum(price.isnull()))

In [ ]:
df_shark = pd.read_csv('./sharks.csv',low_memory=False)
df_shark.head()

In [ ]:
sharkType = df_shark['Type']
sharkType.unique()

sharkType.value_counts()
type = sharkType
type = type.replace(['Sea Disaster','Boat','Boating','Boatomg'],'Watercraft') # Combine all the different watercraft accidents
type.value_counts()

type = type.replace(['Invalid','Questionable','Unconfirmed','Unverified','Under investigation'],np.nan) # Combine all the invalid or N/A accident records
type.value_counts()

df_shark['Type'] = type
del type

df_shark['Type'].value_counts()

In [ ]:
url = 'http://www.vcsc.virginia.gov/pretrialdataproject/October%202017%20Cohort_Virginia%20Pretrial%20Data%20Project_Deidentified%20FINAL%20Update_10272021.csv'
df_pretrial= pd.read_csv(url,low_memory=False)

df_pretrial.head()

release = df_pretrial['WhetherDefendantWasReleasedPretrial']
release = release.replace(9,np.nan) # Remove the nines as it represents unclear data
sum(release.isnull()) # Missing values
df_pretrial['WhetherDefendantWasReleasedPretrial'] = release
del release

In [ ]:
length = df_pretrial['ImposedSentenceAllChargeInContactEvent']
type = df_pretrial['SentenceTypeAllChargesAtConvictionInContractEvent']
length = pd.to_numeric(length,errors='coerce') # Length to numeric
length_NA = length.isnull() # Dummy missing
length = length.mask(type ==4,0) # Replace length with 0
length = length.mask(type ==9, np.nan) # Replace length with N/A
length_NA = length.isnull() # Another dummy missing
df['ImposedSentenceAllChargeInContactEvent'] = length # Replace data with cleaned version
del length,type

**Q3.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?